# imports 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import nonzero
# plt.rcParams.update({'text.usetex':False})

from scipy import stats

import flavio
import flavio.plots
import pandas as pd

import matplotlib.ticker as tck

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from sklearn.metrics import r2_score, accuracy_score

In [5]:
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import optuna
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier

In [2]:
# Functions
def compute_J_from_vec(vec, wilson_coef=None):
    obs_si = ['FL', 'AFB', 'S3', 'S4', 'S5', 'S7', 'S8', 'S9']
    
    if wilson_coef is None:
        print("Setting wilson to sm...")
        #This is the SM
        wilson_coef = flavio.WilsonCoefficients()
        wilson_coef.set_initial({'C9_bsmumu' : 0., 'C10_bsmumu' : 0.}, scale = 100)

    si = {obs: flavio.np_prediction('%s(B0->K*mumu)' % obs, wilson_coef, vec['q2']) for obs in obs_si}

    return compute_J(si, vec['k'], vec['l'], vec['p'])


def compute_J(si, k, l, p):
    ''' Computes the differential branching factor 
    
    si : dict
        dictionary of all S_i terms
    k : float
        theta_k
    l : float
        theta l
    p : float
        phi
    '''

    fl =  3/4*(1 - si['FL']) * (np.sin(k) ** 2) + si['FL'] * (np.cos(k) ** 2) + 1/4 * (1 - si['FL']) * (np.sin(k) ** 2) * np.cos(2*l) - si['FL'] * (np.cos(k) ** 2) * np.cos(2*l)
    s3 = si['S3'] * (np.sin(k) ** 2) * (np.sin(l) ** 2) * np.cos(2 * p)
    s4 = si['S4'] * np.sin(2 * k) * np.sin(2*l) * np.cos(p)
    s5 = si['S5'] * np.sin(2 * k) * np.sin(l) * np.cos(p)
    afb = 4/3 * si['AFB'] * (np.sin(k) ** 2) * np.cos(l)
    s7 = si['S7'] * np.sin(2 * k) * np.sin(l) * np.sin(p)
    s8 = si['S8'] * np.sin(2 * k) * np.sin(2 * l) * np.sin(p)
    s9 = si['S9'] * (np.sin(k) ** 2) * (np.sin(l) ** 2) * np.sin(2 * p)

    return sum([fl, s3, s4, s5, afb, s7, s8, s9])


def compute_J_from_df(df, w):
    obs_si = ['FL', 'AFB', 'S3', 'S4', 'S5', 'S7', 'S8', 'S9']
    
    # si = {obs: flavio.np_prediction('%s(B0->K*mumu)' % obs, w, df['q2']) for obs in obs_si}
    si = {}
    for o in obs_si:
        si[o] = df['q2'].apply(
            lambda q2: flavio.np_prediction(f'{o}(B0->K*mumu)', w, q2)
        )
    si = pd.DataFrame(si)
    return compute_J(si, df['k'], df['l'], df['p'])


def format_range(x, a, b):
    ''' given uniform x in range [0,1], ouptut uniform in range [a,b] '''
    return x * (b - a) + a

# Read Data

In [10]:
# with_weights = pd.read_csv('low_q_with_weights.csv')

f_name = 'toy_data_c9_0.0_c10_0.0_2021_12_6_17.csv'
sm = pd.read_csv('data/' + f_name, index_col=0)
sm.rename(columns={'J_comp':'BR_sm'}, inplace=True)

# Create weight dataset

In [18]:
sm

,q2,k,l,p,BR_sm
0,1.978420,3.019192,1.182624,-0.272753,1.345445
1,0.646961,2.013906,1.686632,1.186758,0.343250
2,1.467531,0.378923,2.140647,1.655859,0.919301
3,1.229330,2.211572,2.994584,-0.109302,0.339375
4,1.778012,1.968873,2.383026,-2.550900,0.405399
...,...,...,...,...,...
15828,0.621425,0.193166,1.736226,-2.433626,0.784051
15829,1.995579,2.364468,2.090352,0.035751,0.665803
15830,0.569111,1.827706,2.399397,0.348114,0.532004
15831,1.661977,2.615980,1.744229,-0.989318,1.091341


# Regression